In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from tqdm.auto import tqdm
from selenium.common.exceptions import NoSuchElementException
from requests.exceptions import MissingSchema

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import os

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:01<00:00, 4.88MB/s]


In [ ]:
# total_link = []
# browser = webdriver.Chrome(service=service, options=chrome_options)
# browser.maximize_window()
# url = f'https://www.google.co.kr/search?q=넷플릭스+다크+리뷰+site:Tistory.com&hl=ko&ei=o_-fZLPaNdLCoASFm5vQBw&start=0&sa=N&ved=2ahUKEwjzt4mUpu3_AhVSIYgKHYXNBno4ChDx0wN6BAgREAI&biw=1152&bih=903&dpr=1.5'
# browser.get(url)
# #browser.implicitly_wait(2)

# for i in range(6): #다음 6번 누름 
#     try:
#         browser.implicitly_wait(2)
#         links = browser.find_elements(By.CSS_SELECTOR, "div.yuRUbf > a")
#         links1 = [_.get_attribute('href') for _ in links]
#         total_link.append(links1)
#         driver = browser.find_element(By.CSS_SELECTOR, '#pnnext > span.SJajHc.NVbCr')
#         time.sleep(1)
#         driver.click()
#     except NoSuchElementException:
#         for i in range(5): #스크롤 5번 내림 
#             browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
#             time.sleep(1)
#             browser.execute_script("return window.scrollY")
#         links = browser.find_elements(By.CSS_SELECTOR, "div.yuRUbf > a")
#         links1 = [_.get_attribute('href') for _ in links]
#         total_link = links1
# total_link = [item for sublist in total_link for item in sublist] #리스트 하나로 풀기
# print(total_link)

In [ ]:
Netflix_df['제목'][:5]

In [3]:
def crawl_tistory_links(keyword_input):
    ''' input에 검색키워드를 넣으면 tistory 후기,리뷰를 검색하여
    url 링크들을 따오는 함수'''

    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.maximize_window()
    url = f"https://www.tistory.com/m/search?query={keyword_input} 리뷰&tab=post"
    browser.get(url)
    browser.implicitly_wait(2)
    
    for i in range(3): #스크롤 3번 내림 
        browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    
    links = browser.find_elements(By.CSS_SELECTOR, "ul.sch_list_post > li > a")
    links1 = [_.get_attribute('href') for _ in links]

    url = f"https://www.tistory.com/m/search?query={keyword_input} 후기&tab=post"
    browser.get(url)
    browser.implicitly_wait(2)
    
    for i in range(3): #스크롤 3번 내림 
        browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    
    links = browser.find_elements(By.CSS_SELECTOR, "ul.sch_list_post > li > a")
    links2 = [_.get_attribute('href') for _ in links]

    url = f"https://www.tistory.com/m/search?query={keyword_input} 감상&tab=post"
    browser.get(url)
    browser.implicitly_wait(2)
    
    for i in range(3): #스크롤 3번 내림 
        browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    
    links = browser.find_elements(By.CSS_SELECTOR, "ul.sch_list_post > li > a")
    links3 = [_.get_attribute('href') for _ in links]
    links = list(set(links1 + links2+ links3))#set으로 중복 링크 제거
    browser.quit()
    
    return links

In [ ]:
# response = requests.get('https://1eed00.tistory.com/287')
# soup = BeautifulSoup(response.text, 'html.parser')
# title = soup.find('div', class_ = 'title_view').h1
# print(title)

In [4]:
def crawl_tistory_blogs(links):
    '''
    crawl_tistory_links로 뽑아온 url을 담은 links를 이용
    BS로 요청하여 title, text를 뽑아옴
    주석처리는 selenium 사용했던 것들
    '''
    crwl_blog_text = []
    titles = []
#    browser = webdriver.Chrome(service=service, options=chrome_options)
#    browser.set_page_load_timeout(7)
    # browser.Manage().Timeouts().PageLoad = TimeSpan.FromMinutes(3)
    tk0 = tqdm(range(len(links)), smoothing=0, mininterval=1.0)
    for i in tk0:
        try:
            # browser.set_page_load_timeout(10)
            blog_text = ''
            # browser.implicitly_wait(2.5)
            #browser.get(links[i])
            
            #WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            response = requests.get(links[i])
            soup = BeautifulSoup(response.text, 'html.parser')
            #soup = BeautifulSoup(browser.page_source, 'html.parser')
            title = soup.find("h3", class_="tit_blogview")            
            if title is None: # 타이틀이 none인 경우 일단 continue해놓음
                print('타이틀이 none', links[i])
                continue
            elif '후기' not in title.text and '리뷰' not in title.text and '감상' not in title.text:#title에 리뷰나 후기라는 단어가 없으면 제거
                continue
            elif keyword not in title.text:
                continue
            div = soup.find("div", class_="blogview_content useless_p_margin editor_ke") # 본문위 class 종류가 2개라 2개 다 불러오고 합침
            div2 = soup.find("div", class_="blogview_content editor_ke")#
            combined_str = str(div) + str(div2)
            combined_soup = BeautifulSoup(combined_str, 'html.parser')
            text = combined_soup.find_all('p')
            text = [_ for _ in text if not _.find('a')]
            for t in text:
                blog_text += t.text
            if blog_text is not None: #본문 text가 존재하면 append 아니면 'None'을 추가
                crwl_blog_text.append(blog_text)
            else:
                crwl_blog_text.append('None')
            if title is not None: #title가 존재하면 append 아니면 'None'을 추가
                titles.append(title.text)
            else:
                titles.append('None')
        except WebDriverException:
            print('WebDriverException에러 발생',links[i])
            pass
        except TimeoutException:
            print('Timeout에러 발생')
            pass
        except requests.exceptions.ConnectionError:
            print("connectionError 발생",links[i])
            pass
        #except MissingSchema:
        #    print("MissingSchema에러 발생", links[i])
        #    pass
    crwl_blog_text = [re.sub('\xa0+|\n+', '',text) for text in crwl_blog_text] #전처리
    
    return crwl_blog_text, titles

In [7]:
Netflix_df = pd.read_csv('C:\\python\\team_project\\finalProject\\netflix_official.csv')
Netflix_df

,Unnamed: 0,제목,장르모음,출연진,개봉일,등급,상영시간,기본장르
0,0,우주 히어로 키드,"키즈 시리즈,가족이 함께 보는 시리즈,코미디 시리즈,만화 시리즈",잭 피셔 톰 케니 어맨다 C. 밀러 킴 야브로 키스 퍼거슨 그레이 그리핀 릴리 로즈...,2021,\n전체관람가,시즌 3개,키즈
1,0,청소년 v 정부: 기후 정의를 외치다,"다큐멘터리 영화,자연 & 생태 다큐멘터리,과학 & 자연 다큐멘터리,사회 & 문화 다...",NaN,2020,\n15+,1시간 49분,다큐멘터리
2,0,하프 배드: 선 & 데빌,"로맨틱한 드라마,드라마,영국 작품,청소년 시리즈,액션 & 어드벤처 시리즈,판타지 시리즈",제이 라이커고 나디아 파크스 에밀리앵 베크만스 폴 레디 이저벨 제스퍼 존스 캐런 코...,2022,\n청불,시즌 1개,판타지
3,0,Breaking Bad,"드라마,범죄 시리즈,미국 TV 프로그램,스릴러 시리즈",브라이언 크랜스턴 에런 폴 안나 건 딘 노리스 벳시 브랜트 RJ 미티 밥 오든커크 ...,2008,\n19+,시즌 5개,드라마 장르
4,0,우리의 지구,"가족이 함께 보는 시리즈,영국 작품,자연 & 생태 다큐멘터리,과학 & 자연 TV 프...",데이비드 애튼버러,2019,\n전체관람가,시즌 2개,다큐멘터리
...,...,...,...,...,...,...,...,...
1798,0,해피 데스데이,"코미디 영화,호러 영화,미스터리 영화,미국 영화",제시카 로스 이즈리얼 브루사드 루비 모딘 찰스 에이킨 로라 클리프턴 제이슨 베일 롭...,2017,\n15+,1시간 36분,호러
1799,0,임금님의 사건수첩,"코미디 영화,스릴러 영화",이선균 안재홍 김희원 주진모 장영남 경수진 정해인 김홍파 김응수,2017,\n12+,1시간 54분,코미디
1800,0,오늘은 회사 쉬겠습니다,"일본 작품,코미디 시리즈,로맨틱 코미디 시리즈,일본 만화 원작 시리즈",아야세 하루카 후쿠시 소타 타마키 히로시 나카 리이사 타구치 준노스케 지바 유다이 ...,2014,\n15+,시즌 1개,로맨스
1801,0,알카티브,"사우디아라비아 작품,중동 영화,드라마 영화,미스터리 영화,사회 이슈 드라마 장르 영화",후삼 알하르티 림 알하비브 누르 알카드라 라하프 이브라힘 아심 알아와드 나디아 알모...,2023,\n15+,1시간 21분,드라마 장르


In [10]:
len(total_text)

285

In [11]:
len(total_title)

285

In [12]:
1512+285

1797

In [17]:
total_title[-1]

['넷플릭스 유 피플 출연진 줄거리 평점 후기 리뷰',
 '영화 [유 피플] 정보 및 등장인물 줄거리 후기',
 '넷플릭스 영화 유 피플 리뷰',
 '영화 유 피플 결말 줄거리 후기 인종차별 로맨틱 코미디 추천?',
 '한 10분 보다보면 뜨악하는 영화 유 피플 리뷰']

In [25]:
a= pd.read_csv('Netflix_tistory_review5.csv')
a

,movie,titles,content
0,데이브레이크,"[""[08.20,21] 데이브레이크의 'SUMMER MADNESS 2016' 공연 ...","['', '(재업)크림에서 데이브레이크를 샀습니다.327이랑 데이브레이크 중에 고민..."
1,비크람: 요가 구루의 두 얼굴,['넷플릭스<비크람: 요가 구루의 두 얼굴> 후기'],['넷플릭스에서 비크람에 관한 다큐멘터리를 봤다. 내가 다니는 요가 학원 수업 중 ...
2,"애슐리 가르시아, 사랑을 배우다",[],[]
3,위기의 레스토랑,['넥플릭스 위기의 레스토랑 리뷰로 하는 영어공부'],"['넥플릭스에 ""위기의 레스토랑""이 인기 콘텐츠에 있더라구요!저는 요리 프로그램을 ..."
4,티퍼니 해디시: 빈민가에서 할리우드로,[],[]
...,...,...,...
280,김종욱 찾기,"['맨 앞 가운데서 봤던 뮤지컬 김종욱 찾기 관람 후기', '[공연/뮤지컬] 김종욱...","['', '', '', '', '', '', '', '\'김종욱 찾기\'의 대만 투..."
281,연애 빠진 로맨스,"['[영화] 연애 빠진 로맨스 후기', '[연애 빠진 로맨스] 손석구X전종서 로맨스...",['애매하다 애매해 ~(코멘트가 아닌 여배우님의 극중 대사)가끔씩은 로맨스 영화를 ...
282,협상,"['법인 세무사 기장료 및 조정료 절감 후기 (feat. 3분 통화로 협상)', '...",['세무사-수수료-기장료현재 총 법인을 2개를 운영하고 있다. 법인 설립시 고정 비...
283,실종,['2022년 12월 6일(화) 미국증시리뷰 - 매수 실종! 하락세가 진정될 때까지...,['1) 이번주 들어와서 증시와 유가 모두 이틀 연속 하락세를 보여주고 있네요. 시...


In [27]:
search_keywords = Netflix_df['제목'][1797:] #검색키워드
total_title = [] #영화별 티스토리 제목들 담는 리스트
total_text = [] #영화별 티스토리 본문을 담는 리스트
tk0 = tqdm(search_keywords, smoothing=0, mininterval=1.0)
for keyword in tk0:
    links = crawl_tistory_links(keyword)
    crwl_blog_text, titles = crawl_tistory_blogs(links)
    total_title.append(titles)
    total_text.append(crwl_blog_text)
df = pd.DataFrame({'movie': Netflix_df['제목'][1797:], 'titles': total_title, 'content': total_text})
df.to_csv('Netflix_tistory_review6.csv',index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0089A813+48355]
	(No symbol) [0x0082C4B1]
	(No symbol) [0x00735358]
	(No symbol) [0x00731B33]
	(No symbol) [0x00728CA5]
	(No symbol) [0x00729D0A]
	(No symbol) [0x00729003]
	(No symbol) [0x00728355]
	(No symbol) [0x007282EC]
	(No symbol) [0x00726F26]
	(No symbol) [0x00727688]
	(No symbol) [0x00736B52]
	(No symbol) [0x0078CF01]
	(No symbol) [0x0077A73C]
	(No symbol) [0x0078C922]
	(No symbol) [0x0077A536]
	(No symbol) [0x007582DC]
	(No symbol) [0x007593DD]
	GetHandleVerifier [0x00AFAABD+2539405]
	GetHandleVerifier [0x00B3A78F+2800735]
	GetHandleVerifier [0x00B3456C+2775612]
	GetHandleVerifier [0x009251E0+616112]
	(No symbol) [0x00835F8C]
	(No symbol) [0x00832328]
	(No symbol) [0x0083240B]
	(No symbol) [0x00824FF7]
	BaseThreadInitThunk [0x778A00C9+25]
	RtlGetAppContainerNamedObjectPath [0x779E7B4E+286]
	RtlGetAppContainerNamedObjectPath

In [18]:
a = pd.read_csv('Netflix_tistory_review2.csv')

In [32]:
len(Netflix_df['제목'][1400:1513])

113

In [33]:
df = pd.DataFrame({'movie': Netflix_df['제목'][1400:1512], 'titles': total_title, 'content': total_text})
df.to_csv('Netflix_tistory_review4.csv',index=False)

In [34]:
df

,movie,titles,content
1400,위험한 만찬,"[[영화 한 줄 리뷰] 위험한 만찬, nothing to hide(위험한 만찬) 후...",[ 한 줄 리뷰 : 엇박의 긴장감이 주는 불편한 스릴. 추천/비추천 : 추천 개봉일...
1401,아파야 사는 남자,[],[]
1402,베이비,"[액션영화 '베이비 드라이버 2017' 줄거리 ost 음악 자동차 리뷰, 노부영 베...",['베이비 드라이버'는 전 세계적으로 큰 인기를 끌었던 액션 영화입니다. 영화의 주...
1403,1983,"[넷플릭스 <1983> 평점/리뷰/해외반응, [역곡/1983피자앤펍] 백종원의 골목...","[, 안녕하세요~ 난날리뷰의 난날입니다 ๑’ٮ’๑ 오늘 포스팅 할 곳은 역곡에 위치..."
1404,나이트폴: 신의 기사단,[],[]
...,...,...,...
1507,당신과 자연의 대결,[],[]
1508,파밀리예,[],[]
1509,트웰브는 영원히,[],[]
1510,남편의 그것이 들어가지 않아,[넷플릭스 <남편의 그것이 들어가지 않아> 평점/리뷰/해외반응],[]
